In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression

#Finding which stocks have an anomaly

openingPrices = pd.read_excel("data.xlsx", sheet_name = 'OpeningPrice', skiprows = [0]) # 30 stocks opening prices
closingPrices = pd.read_excel("data.xlsx", sheet_name = 'ClosingPrice', skiprows = [0]) # 30 stocks closing prices

openingPrices["Date"] = pd.to_datetime(openingPrices.Date, format='%Y-%m-%d')
closingPrices["Date"] = pd.to_datetime(closingPrices.Date,format='%Y-%m-%d')

openingPrices["Weekday"] = pd.to_datetime(openingPrices.Date).dt.dayofweek # adding column to get track of which day of the week
closingPrices["Weekday"] = pd.to_datetime(closingPrices.Date).dt.dayofweek

first_beg = pd.to_datetime("2000-02-07",format='%Y-%m-%d') #begining period of 1st sample
first_end = pd.to_datetime("2007-12-31",format='%Y-%m-%d') #end period -- 

second_beg = pd.to_datetime("2009-01-01",format='%Y-%m-%d') #begining period of 2nd sample
second_end = pd.to_datetime("2019-12-31",format='%Y-%m-%d') #end period --

openingPrices.columns = ['Date','AAPL','MSFT', 'JPM', 'JNJ', 'AMZN', 'BAC', 'PG', 'XOM', 'INTC','T','KO','UNH','DIS','HD','VZ','DY','MRK','CVX','PFE','WFC','CSCO','PEP','CMCSA','BA','C','ORCL','MCD','ABT','IBM','NVDA','Weekday']
closingPrices.columns = ['Date','AAPL','MSFT', 'JPM', 'JNJ', 'AMZN', 'BAC', 'PG', 'XOM', 'INTC','T','KO','UNH','DIS','HD','VZ','DY','MRK','CVX','PFE','WFC','CSCO','PEP','CMCSA','BA','C','ORCL','MCD','ABT','IBM','NVDA','Weekday']

### Question 1
Using the closing prices the following was computed for all 30 stocks:
- the average return
- the standard deviation
- the Sharpe ratio

In [2]:
ret_assets = closingPrices.iloc[:,1:31].pct_change() #ignoring the first columns (date) compute the daily change
mean_ret_assets = ret_assets.mean() # avergage daily return of each asset
std_assets = ret_assets.std() # volatilitiy, standard deviation of return
sharpe_assets = mean_ret_assets / std_assets * np.sqrt(250) # Annualized Sharpe Ratio

# uncomment to see results
# print(mean_ret_assets)
# print(std_assets)
# print(sharpe_assets)

### Question 2 & 4
 - Implementation of the weekend anomaly for Monday (i.e day == 0)
 - Implementation of anomaly search for Tuesday through Friday (i.e day > 0)

In [3]:
#analysis: returns a dictionary ans
def analysis(returns):
    return_avg = returns.mean()
    return_vol = returns.std()
    return_sharpe = return_avg / return_vol * np.sqrt(250) #how to annualize?   
    significant = []
    pvalue_df = return_avg.copy() # initialize pvalue
    X = []
    y = []
    
    # for each stock we want to compute its pvalue to determine if its significantly different from 0
    for stock in pvalue_df.index:
        data_reg = returns
        data_reg['Const'] = 1
        data_reg.dropna(inplace = True)
        
        X = data_reg.Const
        y = data_reg[stock] * 250
        model = sm.OLS(y, X).fit() 
        conf_int = model.conf_int(alpha = 0.05)
        
        pvalue_df[stock] = model.pvalues
        if(pvalue_df[stock] <= 0.05):
            significant.append(stock)
            
    #storing values to the dictionaries
    ans = {
        'Average': return_avg,
        'Volatility': return_vol,
        'Sharpe': return_sharpe,
        'Pvalue': pvalue_df,
        'Significant': significant
    }
    return ans

In [4]:
ans = {} # dict to store {'Average': _, 'Volatility': _, 'Sharpe': _, 'Pvalue': _}
answers = {} # each day of the week we record the ans; {Monday: ans, Tuesday: ans...}
day = 0 # starting with Monday, 1 Tuesday and onwards

while day <= 4:
    ans = {}
    Closing = closingPrices[closingPrices.Weekday == day].iloc[:,1:31] #stock prices for given day
    Opening = openingPrices[openingPrices.Weekday == day].iloc[:,1:31] 
    
    # Monday we short, everyother day we go long
    if day == 0: 
        returns = (Opening - Closing)/Opening # sell at opening, buy closing
    else:
        returns = (Closing - Opening)/Opening # buy at opening, sell closing
    
    ans = analysis(returns)
    answers.update({day:ans})
    
    print("Significant stocks for day %d are the following :"%(day))
    print(answers[day]['Significant'])
    print("")
    
    # for all stocks information replace for loop with: for stock in closingPrices.columns[1:31]:
    for stock in answers[day]['Significant']:
        print(stock)
        print("Average: %f"%answers[day]['Sharpe'][stock])
        print("Volatilitiy: %f"%answers[day]['Volatility'][stock])
        print("Sharpe: %f"%answers[day]['Sharpe'][stock]) 
        print("Pvalue: %f"%answers[day]['Pvalue'][stock])
        print("")
    
    day = day + 1 # go to the next day of the week
    

Significant stocks for day 0 are the following :
['AAPL', 'PG', 'PEP', 'C', 'IBM']

AAPL
Average: -1.339450
Volatilitiy: 0.021621
Sharpe: -1.339450
Pvalue: 0.009467

PG
Average: -1.334353
Volatilitiy: 0.010349
Sharpe: -1.334353
Pvalue: 0.009742

PEP
Average: -1.286227
Volatilitiy: 0.010092
Sharpe: -1.286227
Pvalue: 0.012704

C
Average: 1.036057
Volatilitiy: 0.023917
Sharpe: 1.036057
Pvalue: 0.044597

IBM
Average: -1.514878
Volatilitiy: 0.012852
Sharpe: -1.514878
Pvalue: 0.003356

Significant stocks for day 1 are the following :
['JNJ', 'PG', 'XOM', 'KO', 'PEP', 'ABT', 'IBM']

JNJ
Average: 1.176557
Volatilitiy: 0.009721
Sharpe: 1.176557
Pvalue: 0.018152

PG
Average: 1.402459
Volatilitiy: 0.010936
Sharpe: 1.402459
Pvalue: 0.004720

XOM
Average: 1.120145
Volatilitiy: 0.012840
Sharpe: 1.120145
Pvalue: 0.021189

KO
Average: 1.155297
Volatilitiy: 0.010524
Sharpe: 1.155297
Pvalue: 0.018777

PEP
Average: 1.319041
Volatilitiy: 0.010450
Sharpe: 1.319041
Pvalue: 0.005961

ABT
Average: 0.962409
Vo

### Question 3
- Sample 1 from Feb 7, 2000 - Dec 31, 2007
- Sample 2 from Jan 1, 2009 - Dec 31, 2019

In [5]:
# 2 samples seperated, compare first with whole & second with whole

#seperate sample based on dates, open or close prices
first_open = openingPrices.loc[(openingPrices.Date>=first_beg) & (openingPrices.Date<=first_end),
                               openingPrices.columns].copy()
first_close = closingPrices.loc[(closingPrices.Date>=first_beg) & (closingPrices.Date<=first_end),
                                closingPrices.columns].copy()

second_open = openingPrices.loc[(openingPrices.Date>=second_beg) & (openingPrices.Date<=second_end),
                                openingPrices.columns].copy()
second_close = closingPrices.loc[(closingPrices.Date>=second_beg) & (closingPrices.Date<=second_end),
                                 closingPrices.columns].copy()

#then further seperate sample and pick only data with dates on a Monday
first_openM = first_open[first_open.Weekday == 0].iloc[:,1:31] 
first_closeM = first_close[first_close.Weekday == 0].iloc[:,1:31]

second_openM = second_open[second_open.Weekday == 0].iloc[:,1:31]
second_closeM = second_close[second_close.Weekday == 0].iloc[:, 1:31]

# compute returns for first and second sample
first_returns = (first_openM - first_closeM)/first_openM 
second_returns = (second_closeM - second_openM)/second_openM

first_ans = analysis(first_returns)
second_ans = analysis(second_returns)

print(answers[0]['Significant'])
print("")

# for all stocks information replace for loop with: for stock in closingPrices.columns[1:31]:
print("first sample")
print(first_ans['Significant'])
for stock in first_ans['Significant']:
    print(stock)
    print("Average: %f"%first_ans['Average'][stock])
    print("Volatilitiy: %f"%first_ans['Volatility'][stock])
    print("Sharpe: %f"%first_ans['Sharpe'][stock]) 
    print("Pvalue: %f"%first_ans['Pvalue'][stock])
    print("")

print("second")
print(second_ans['Significant'])
for stock in second_ans['Significant']:
    print(stock)
    print("Average: %f"%second_ans['Average'][stock])
    print("Volatilitiy: %f"%second_ans['Volatility'][stock])
    print("Sharpe: %f"%second_ans['Sharpe'][stock]) 
    print("Pvalue: %f"%second_ans['Pvalue'][stock])
    print("")

['AAPL', 'PG', 'PEP', 'C', 'IBM']

first sample
['BAC', 'PG', 'MRK', 'PEP', 'MCD', 'IBM']
BAC
Average: -0.001741
Volatilitiy: 0.013435
Sharpe: -2.049330
Pvalue: 0.012616

PG
Average: -0.001541
Volatilitiy: 0.012043
Sharpe: -2.022538
Pvalue: 0.013813

MRK
Average: -0.001587
Volatilitiy: 0.013968
Sharpe: -1.796201
Pvalue: 0.028638

PEP
Average: -0.001853
Volatilitiy: 0.011639
Sharpe: -2.516883
Pvalue: 0.002235

MCD
Average: -0.002142
Volatilitiy: 0.015648
Sharpe: -2.164794
Pvalue: 0.008446

IBM
Average: -0.001715
Volatilitiy: 0.015719
Sharpe: -1.724767
Pvalue: 0.035560

second
['AAPL', 'CSCO', 'IBM']
AAPL
Average: 0.002418
Volatilitiy: 0.013988
Sharpe: 2.732760
Pvalue: 0.000092

CSCO
Average: 0.001192
Volatilitiy: 0.011082
Sharpe: 1.700843
Pvalue: 0.014495

IBM
Average: 0.001158
Volatilitiy: 0.009594
Sharpe: 1.908557
Pvalue: 0.006120

